In [19]:
import re
import datetime
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

### Get Information from Property Rent Page

In [21]:
def house_info(non_ad_houses):
    prop_features = ['bed','shower','']
    house_urls  = []
    house_names = []
    house_price = []
    house_beds  = []
    house_showers = []
    house_garages = []
    house_area = []
    #house_description
    house_currency = []
    house_rent_period = []
    house_address = []
    #house_time_posted = []
    for link in non_ad_houses:
        house_names.append(link.text.split(',')[0].title().strip())
        house_urls.append(base_url + link.find('a').attrs['href'])
        house_address.append(link.text.split(',')[0].title().split('At ')[-1])
        price = link.find('p', {'class':'h3'}).text.replace('\n','').replace('Price','').replace('$','').replace('GH₵','').split('/')[0].strip()
        currency_temp = link.find('p', {'class':'h3'}).text.replace('\n','').replace('Price','').split('/')[0].strip()
        currency = currency_temp[0] if currency_temp[0] is '$' else 'GHC'
        beds =   link.find('li', {'class':'bed'}).text if link.find('li', {'class':'bed'}) is not None else ''
        showers =   link.find('li', {'class':'shower'}).text if link.find('li', {'class':'shower'}) is not None else ''
        area = link.find('li', {'title':'Area'}).text if link.find('li', {'title':'Area'}) is not None else ''
        garages = link.find('li', {'title':'Garages'}).text if link.find('li', {'title':'Garages'}) is not None else ''
        period =  link.find('p', {'class':'h3'}).text.split('/ ')[-1].strip()
        
              
        house_beds.append(beds)
        house_showers.append(showers)
        house_price.append(price)
        house_currency.append(currency)
        house_garages.append(garages)
        house_area.append(area)
        house_rent_period.append(period)
    
    df_houses = pd.DataFrame({'Property': house_names, 
                              'Beds': house_beds,
                              'Currency' : house_currency,
                              'price': house_price, 
                              'garages': house_garages, 
                              'Link': house_urls,
                              'address': house_address,
                              'rent_period': house_rent_period, 
                              'area': house_area,
                              'address': house_address
                              })
    
    return(df_houses)


def next_page(page_num, property_url, requests):
    next_url = property_url + '?w=' + str(page_num)
    result = requests.get(next_url)
    if result.status_code == 200:
        print('Scraping page number:', page_num)
    else:
        print('Error occured in page load')
        
    return result, requests, next_url



def scrape_newest_listing(df_saved_listings):
    current_page = requests.get('https://meqasa.com/properties-for-rent-in-ghana')
    src = current_page.content
    soup = BeautifulSoup(src, 'lxml')
    
    non_ad_houses  = soup.findAll('div', {'class': 'mqs-prop-dt-wrapper'})
    df_non_ad_house = house_info(non_ad_houses)
    

def save_data(df):
    file_time = datetime.datetime.now()
    file_time = file_time.strftime('%Y-%m-%d')
    fine_name = 'meqasa_'+file_time+'.csv'
    
    df.to_csv('data/'+fine_name, index=False)
    
    
        
        
        
        
        
    

In [22]:
df_final = pd.DataFrame([])
df_first_ad = pd.DataFrame([])

base_url = 'https://meqasa.com'
result   = requests.get('https://meqasa.com/properties-for-rent-in-ghana')
property_url = result.url

# Enter the number of pages you would like to scrape
number_of_pages = 2

print('========================')
print('==== Scrape Meqasa ====')

for i in range(1, number_of_pages+1):
    df_first_ad = pd.DataFrame([])
    
    result, requests, next_url = next_page(i, property_url, requests)
    src  = result.content
    soup = BeautifulSoup(src, 'lxml')
    
    #first_ad_house = soup.findAll('div', {'class': 'mqs-prop-dt-wrapper'})
    non_ad_houses  = soup.findAll('div', {'class': 'mqs-prop-dt-wrapper'})
    
    #df_first_ad = house_info(first_ad_house)
    df = house_info(non_ad_houses)
    
    df_final = df_final.append(df)
  
    
    df_final.reset_index(drop=True, inplace=True)
    


# Add the listing time and listing broker
print('Now gathering additional information (Broker and Listing Time)...')

#Newest_listing_time = df_final['Listing Time'][0]

print('\n')
#print('Time of last listing posted:', Newest_listing_time)
    
save_data(df_final)
print('Scaping Complete')

==== Scrape Meqasa ====
Scraping page number: 1
Scraping page number: 2
Now gathering additional information (Broker and Listing Time)...


Scaping Complete
